In [1]:
################################################################################
import os
import sys
import pyspark
from pyspark.sql import SparkSession
from delta.tables import *
from delta.tables import DeltaTable
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql.functions import from_unixtime, col, to_timestamp
from pyspark.sql.functions import udf
import hashlib
import urllib.request
import json
from datetime import timedelta, date
from pyspark import SparkContext
from pyspark import SQLContext
from itertools import islice
from pyspark.sql.functions import col
################################################################################

In [2]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import os


def download_path(filename):
    """
    Args:
        filename: name of the file located in s3 bucket that will be downloaded
    
    output:
        downloads a specific filename from s3 and saves it to a local directory called data
        for staging
    """
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    bucket_name = 'd2b-internal-assessment-bucket'
    response = s3.list_objects(Bucket=bucket_name, Prefix="orders_data")
    # specify directory to save downloaded files
    local = '/home/jovyan/work/data'
    if not os.path.exists(local):
       os.makedirs(local)

    s3.download_file(bucket_name, "orders_data/{}".format(filename), local + "/" "{}".format(filename))


if __name__ == "__main__":
    download_path("orders.csv")
    download_path("reviews.csv")
    download_path("shipment_deliveries.csv")

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0,org.apache.hadoop:hadoop-client:3.2.0")
    .config("spark.jars.excludes", "com.google.guava:guava")
    .getOrCreate()
)

# Replace "s3://d2b-internal-assessment-bucket" with the path to your S3 bucket
df = spark.read.format("csv").option("header", True).load("s3://d2b-internal-assessment-bucket/orders_data/orders.csv")

df.show()

:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-edc9e336-abc1-403a-aa31-40bca1bd4320;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
	found org.apache.hadoop#hadoop-client;3.2.0 in central
	found org.apache.hadoop#hadoop-common;3.2.0 in central
	found org.apache.hadoop#hadoop-annotations;3.2.0 in central
	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found org.apache.httpcomponents#httpclient;4.5.2 in central
	found org.apache.httpcomponents#httpcore;4.4.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.11 in central
	found commons-io#commons-io;2.5 in centr

Py4JJavaError: An error occurred while calling o52.load.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:747)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:745)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNecessary(Dat

In [7]:
################################################################################
# Define schema for orders data
orders_schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("order_date", DateType(), True),
    StructField("product_id", IntegerType(), True),
    StructField("unit_price", IntegerType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("total_price", IntegerType(), True)
])
################################################################################
# Create Spark session
spark = SparkSession.builder.appName("S3ToPostgres").getOrCreate()
################################################################################
# Set AWS credentials
spark.conf.set("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com")
spark.conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
################################################################################
# Read orders data from S3
bucket_name = "d2b-internal-assessment-bucket"
prefix = "orders_data"
table_name = "orders"
orders_df = spark.read.schema(orders_schema).csv(f"s3://{bucket_name}/{prefix}/{table_name}.csv")

23/04/24 14:29:09 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3://d2b-internal-assessment-bucket/orders_data/orders.csv.
org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:370)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(

Py4JJavaError: An error occurred while calling o133.csv.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:747)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:745)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNecessary(Dat